In [3]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import KFold
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score


%matplotlib inline
%config InlineBackend.figure_formats = {'png', 'retina'}
pd.options.mode.chained_assignment = None  # default='warn'?

# data_key DataFrame
data_key = pd.read_csv('key.csv')

# data_train DataFrame
data_train = pd.read_csv('train.csv')

# data_weather DataFrame
data_weather = pd.read_csv('weather.csv')

rain_text = ['FC', 'TS', 'GR', 'RA', 'DZ', 'SN', 'SG', 'GS', 'PL', 'IC', 'FG', 'BR', 'UP', 'FG+']
other_text = ['HZ', 'FU', 'VA', 'DU', 'DS', 'PO', 'SA', 'SS', 'PY', 'SQ', 'DR', 'SH', 'FZ', 'MI', 'PR', 'BC', 'BL', 'VC' ]

data_weather['codesum'].replace("+", "")
a = []
for i in range(len(data_weather['codesum'])):
    a.append(data_weather['codesum'].values[i].split(" "))
    for i_text in a[i]:
        if len(i_text) == 4:
            a[i].append(i_text[:2])
            a[i].append(i_text[2:])
            
data_weather["nothing"] = 1
data_weather["rain"] = 0
data_weather["other"] = 0
b = -1
for ls in a:
    b += 1
    for text in ls:
        if text in rain_text:
            data_weather.loc[b, 'rain'] = 1
            data_weather.loc[b, 'nothing'] = 0
        elif text in other_text:
            data_weather.loc[b,'other'] = 1
            data_weather.loc[b, 'nothing'] = 0      
# 모든 데이터 Merge
df = pd.merge(data_weather, data_key)

station_nbr = df['station_nbr']
df.drop('station_nbr', axis=1, inplace=True)
df['station_nbr'] = station_nbr

df = pd.merge(df, data_train)

# T 값 처리 하기. Remained Subject = > 'M' and '-'
df['snowfall'][df['snowfall'] == '  T'] = 0.05
df['preciptotal'][df['preciptotal'] == '  T'] = 0.005

# 주말과 주중 구분 작업 하기
df['date'] = pd.to_datetime(df['date'])
df['week7'] = df['date'].dt.dayofweek
df['weekend'] = 0
df.loc[df['week7'] == 5, 'weekend'] = 1
df.loc[df['week7'] == 6, 'weekend'] = 1

df1 = df[df['station_nbr'] == 1]; df11 = df[df['station_nbr'] == 11]
df2 = df[df['station_nbr'] == 2]; df12 = df[df['station_nbr'] == 12]
df3 = df[df['station_nbr'] == 3]; df13 = df[df['station_nbr'] == 13]
df4 = df[df['station_nbr'] == 4]; df14 = df[df['station_nbr'] == 14]
df5 = df[df['station_nbr'] == 5]; df15 = df[df['station_nbr'] == 15]
df6 = df[df['station_nbr'] == 6]; df16 = df[df['station_nbr'] == 16]
df7 = df[df['station_nbr'] == 7]; df17 = df[df['station_nbr'] == 17]
df8 = df[df['station_nbr'] == 8]; df18 = df[df['station_nbr'] == 18]
df9 = df[df['station_nbr'] == 9]; df19 = df[df['station_nbr'] == 19]
df10 = df[df['station_nbr'] == 10]; df20 = df[df['station_nbr'] == 20]

In [8]:
df14 = df14.apply(pd.to_numeric, errors = 'coerce')
df14.describe().iloc[:, 14:]
# 없는 Column = date, codesum, depart, sunrise, sunset, station_nbr

,stnpressure,sealevel,resultspeed,resultdir,avgspeed,nothing,rain,other,store_nbr,station_nbr,item_nbr,units,week7,weekend
count,386280.000000,382728.000000,387612.000000,387612.000000,387612.000000,388500.000000,388500.000000,388500.000000,388500.000000,388500.0,388500.000000,388500.000000,388500.000000,388500.000000
mean,28.627805,30.000731,9.326919,17.941581,11.231730,0.651429,0.326857,0.101714,22.000000,14.0,56.000000,1.340535,2.994286,0.285714
std,0.172780,0.192018,4.867972,9.656188,4.407667,0.476519,0.469065,0.302273,18.110794,0.0,32.041681,11.241810,2.005416,0.451755
min,28.040000,29.390000,0.500000,1.000000,2.400000,0.000000,0.000000,0.000000,2.000000,14.0,1.000000,0.000000,0.000000,0.000000
25%,28.510000,29.870000,5.700000,14.000000,7.700000,0.000000,0.000000,0.000000,5.000000,14.0,28.000000,0.000000,1.000000,0.000000
50%,28.620000,29.980000,8.700000,17.000000,10.500000,1.000000,0.000000,0.000000,22.000000,14.0,56.000000,0.000000,3.000000,0.000000
75%,28.740000,30.120000,12.500000,21.000000,14.200000,1.000000,1.000000,0.000000,39.000000,14.0,84.000000,0.000000,5.000000,1.000000
max,29.280000,30.690000,26.000000,36.000000,26.100000,1.000000,1.000000,1.000000,42.000000,14.0,111.000000,343.000000,6.000000,1.000000


In [6]:
df14['store_nbr'].unique()

array([ 2,  6, 38, 42], dtype=int64)

In [9]:
df14_drop_columns = ['date', 'codesum', 'station_nbr']
df14 = df14.drop(columns = df14_drop_columns)

In [10]:
# np.nan를 포함하고 있는 변수(column)를 찾아서, 그 변수에 mean 값 대입해서 Frame의 모든 Value가 fill 되게 하기.
df14_columns = df14.columns

# Cateogry 값을 포함하는 변수는 np.nan에 mode값으로 대체하고, 나머지 실수 값을 포함한 변수는 np.nan에 mean값으로 대체
for i in df14_columns:
    if (i == 'resultdir'):
        df14[i].fillna(df14[i].mode()[0], inplace=True)
        print(df14[i].mode()[0])
    else:
        df14[i].fillna(df14[i].mean(), inplace=True)

# 이제 모든 변수가 숫자로 표기 되었기 때문에, 가능 함. 
# 상대 습도 추가 #
df14['relative_humility'] = 100*(np.exp((17.625*((df14['dewpoint']-32)/1.8))/(243.04+((df14['dewpoint']-32)/1.8)))/np.exp((17.625*((df14['tavg']-32)/1.8))/(243.04+((df14['tavg']-32)/1.8))))

# 체감온도 계산
df14["windchill"] = 35.74 + 0.6215*df14["tavg"] - 35.75*(df14["avgspeed"]**0.16) + 0.4275*df14["tavg"]*(df14["avgspeed"]**0.16)

df14 = df14[df14['units'] != 0]

17.0


In [11]:
# 'np.log1p(units) ~ tmax + tmin + tavg + depart + dewpoint + wetbulb + heat + cool + sunrise + sunset + codesume + \
# snowfall + preciptotal + stnpressure + sealevel + resultspeed + resultdir + avgspeed + nothing + rain + other + \
# store_nbr + station_nbr + item_nbr'

model_df14 = sm.OLS.from_formula('np.log1p(units) ~ depart + sunset + sunrise + tmax + tmin + tavg + dewpoint + heat + cool + preciptotal + \
resultspeed + sealevel + snowfall + resultdir + avgspeed + C(nothing) + C(rain) + C(other) + C(item_nbr) + C(week7) + \
C(weekend) + relative_humility + windchill + 0', data = df14)
result_df14 = model_df14.fit()

print(result_df14.summary())

                            OLS Regression Results                            
Dep. Variable:        np.log1p(units)   R-squared:                       0.888
Model:                            OLS   Adj. R-squared:                  0.888
Method:                 Least Squares   F-statistic:                     1925.
Date:                Wed, 04 Jul 2018   Prob (F-statistic):               0.00
Time:                        16:38:41   Log-Likelihood:                -6041.5
No. Observations:                9017   AIC:                         1.216e+04
Df Residuals:                    8979   BIC:                         1.243e+04
Df Model:                          37                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
C(nothing)[0]         -0.0344      0

In [12]:
model_df14 = sm.OLS.from_formula('np.log1p(units) ~ scale(depart) + scale(sunset) + scale(sunrise) + scale(tmax) + scale(tmin) + scale(tavg) + scale(dewpoint) + scale(heat) + scale(cool) + \
scale(preciptotal) + scale(resultspeed) + scale(sealevel) + scale(snowfall)+ scale(resultdir) + scale(avgspeed) + C(nothing) + C(rain) + \
C(other) + C(item_nbr) + C(week7) + C(weekend) + scale(relative_humility) + scale(windchill) + 0', data = df14)
result_df14 = model_df14.fit()

print(result_df14.summary())

                            OLS Regression Results                            
Dep. Variable:        np.log1p(units)   R-squared:                       0.888
Model:                            OLS   Adj. R-squared:                  0.888
Method:                 Least Squares   F-statistic:                     1925.
Date:                Wed, 04 Jul 2018   Prob (F-statistic):               0.00
Time:                        16:39:04   Log-Likelihood:                -6041.5
No. Observations:                9017   AIC:                         1.216e+04
Df Residuals:                    8979   BIC:                         1.243e+04
Df Model:                          37                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
C(nothing)[0]           

In [13]:
anova_result_df14 = sm.stats.anova_lm(result_df14).sort_values(by=['PR(>F)'], ascending = False)
anova_result_df14[anova_result_df14['PR(>F)'] <= 0.05]

C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


,df,sum_sq,mean_sq,F,PR(>F)
scale(resultspeed),1.0,0.946230,0.946230,4.213752,4.012646e-02
scale(windchill),1.0,1.210190,1.210190,5.389217,2.028382e-02
scale(relative_humility),1.0,1.333476,1.333476,5.938238,1.483511e-02
scale(tmin),1.0,2.691900,2.691900,11.987572,5.380605e-04
scale(sunrise),1.0,2.734540,2.734540,12.177455,4.860267e-04
C(other),1.0,5.665046,5.665046,25.227589,5.192899e-07
scale(heat),1.0,7.727095,7.727095,34.410305,4.621481e-09
C(week7),6.0,108.737843,18.122974,80.705242,1.030581e-98
C(item_nbr),12.0,15860.562190,1321.713516,5885.855778,0.000000e+00
C(nothing),2.0,107939.115502,53969.557751,240337.281502,0.000000e+00


In [14]:
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(df14.values, i) for i in range(df14.shape[1])]
vif["features"] = df14.columns
vif = vif.sort_values("VIF Factor").reset_index(drop=True)
vif
# 10순위까지 겹치는것만 쓴다
# C(other), C(week7), C(item_nbr), 

,VIF Factor,features
0,1.032188,store_nbr
1,1.059579,item_nbr
2,1.090404,units
3,1.130201,snowfall
4,1.195896,resultdir
5,1.237218,preciptotal
6,1.819570,other
7,2.600252,depart
8,2.757445,week7
9,2.764910,weekend


In [15]:
# C(other), C(week7), C(item_nbr), 
model_df14 = sm.OLS.from_formula('np.log1p(units) ~ C(item_nbr) + C(other) + C(week7) + 0', data = df14)
result_df14 = model_df14.fit()

print(result_df14.summary())

                            OLS Regression Results                            
Dep. Variable:        np.log1p(units)   R-squared:                       0.887
Model:                            OLS   Adj. R-squared:                  0.887
Method:                 Least Squares   F-statistic:                     3709.
Date:                Wed, 04 Jul 2018   Prob (F-statistic):               0.00
Time:                        16:40:56   Log-Likelihood:                -6092.4
No. Observations:                9017   AIC:                         1.222e+04
Df Residuals:                    8997   BIC:                         1.237e+04
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
C(item_nbr)[5]       3.8275      0.017  

In [16]:
# resultspeed, avgspeed, C(weekend), preciptotal, C(other), C(week7), C(item_nbr),
X14 = df14[['other', 'week7', 'item_nbr']]
y14 = df14['units']
model14 = LinearRegression()

cv14 = KFold(n_splits=14, shuffle=True, random_state=0)

cross_val_score(model14, X14, y14, scoring="r2", cv=cv14)

array([0.04005985, 0.03890528, 0.05331831, 0.04898802, 0.03036825,
       0.0566353 , 0.05246565, 0.06150363, 0.04888226, 0.05400796,
       0.0451179 , 0.01704434, 0.06227728, 0.04106625])